In [74]:
#Importing libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

#Importing webpage using BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text  
soup = BeautifulSoup(source, 'lxml')
soup

#Finding the table
table1 = soup.find('table', {'class':'wikitable sortable'})

#Extracting each row
table_rows= table1.find_all('tr')
table_rows

#Create an empty list
list_postal_codes= []  # "pd.DataFrame(columns=['PostalCode', 'Borough','Neighborhood'])

#Reading attributes of each row 
for tr in table_rows:
    td = tr.find_all('td')
#Controlling empty rows
    if len(td) == 0:
        pass
    else:
#Extracting text of each attribute and processing cells without "Not assigned" Borough
        row= [i.text for i in td]
        if row[1] == 'Not assigned':
            pass
        
        else:
            
#Changing "Not assigned" in Neighborhood for the same value as Borough
            if 'Not assigned' in row[2]:
                row[2] = row[1]

#Replacing '\n' in Neighborhood for ''
            if '\n' in row[2]:
                row[2]= row[2].rstrip() #Remove '\n'
                row[2]= row[2]+', ' #Adding ', ' before applying groupby

#Adding the row to the list 'list_postal_codes'
            list_postal_codes.append(row)


#Creating Dataframe
PostalCodes = pd.DataFrame(data=list_postal_codes, columns=['PostalCode', 'Borough','Neighborhood'])
PostalCodes

#Grouping by PostalCode
df = PostalCodes.groupby(['PostalCode', 'Borough']).sum()
df= df.reset_index()
df['second'] = ""
#Cleaning data. Removing ', ' from the last two characters in Neighborhood
strlen = 0
for n in df['Neighborhood']:
    strlen= len(n)
    n = n[0:strlen - 2]
    df['Neighborhood'][0] = n
    #print(newitem)
    
df.head()

,PostalCode,Borough,Neighborhood,second
0,M1B,Scarborough,Northwest,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union,",
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,",
3,M1G,Scarborough,"Woburn,",
4,M1H,Scarborough,"Cedarbrae,",


In [44]:
#Dataframe shape
df.shape

(103, 3)